In [1]:
import torch
from collections import Counter
from torch.utils.data import DataLoader
from torchsummary import summary
import numpy as np
from torch import nn, optim
import torch.nn.functional as F


In [2]:

class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}
        self.words_indexes = [self.word_to_index[w] for w in self.words]
        self.sequence_length=4
    def load_words(self):
        self.text= open("lyrics2.txt","r")
        self.text=str(self.text.read())
        return self.text.split(' ')
    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)
    def __len__(self):
        return len(self.words_indexes) - self.sequence_length
    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

In [3]:
dataset=Dataset()
dataloader = DataLoader(dataset, batch_size=512)


In [4]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc1 = nn.Linear(self.lstm_size, 50408)
        self.fc = nn.Linear(50408, n_vocab)


    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = F.tanh(self.fc1(output))
        logits = (self.fc(logits))

        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [ ]:
model=Model(dataset)
print(model.cuda())

In [1]:
model.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
        state_h, state_c = model.init_state(4)
        state_h, state_c= state_h.cuda(), state_c.cuda()
        for batch, (x, y) in enumerate(dataloader):
            x,y=x.cuda(),y.cuda()
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            if (batch%256==0):
                print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
                torch.save(model.state_dict(), "poetry.pt")

NameError: name 'model' is not defined

In [2]:
model.load_state_dict(torch.load("poetry.pt"))

NameError: name 'model' is not defined

In [3]:
def predict(dataset, model, text, next_words=50):
    words = text.split(' ')
    model.eval()

    state_h, state_c = model.init_state(len(words))
    state_h, state_c= state_h.cuda(), state_c.cuda()


    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]]).cuda()
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        word_weights = torch.nn.functional.softmax(y_pred.squeeze().data.div(0.64).exp().cpu(),dim=0).cpu().detach()
        word_idx = torch.multinomial(word_weights, 1)[0]
        #print(word_idx)
        words.append(dataset.index_to_word[word_idx.item()])
    return (words)
    


In [4]:
word_list = (predict(dataset, model, text='zindagi'))
strword=str()
for i in word_list:
    strword+=" "+i
print(strword)

NameError: name 'dataset' is not defined